# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import time
from itertools import islice
from dataclasses import dataclass
import torchvision
from torchvision.models import densenet161, DenseNet161_Weights, vit_b_16, ViT_B_16_Weights, densenet121, DenseNet121_Weights
import os
import sys
from pathlib import Path
from torchinfo import summary

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
from CheXpert.race_prediction.dataset import CheXpertRaceDataset
from CheXpert.disease_prediction.dataset import CheXpertDiseaseDataset
from shared_utils import vprint, to_gpu, add_mean_to_list, Mode, SharedConfigs
import shared_utils
from CheXpert.disease_prediction.utils import Configs as DiseaseConfigs
from CheXpert.race_prediction.utils import Configs as RaceConfigs
from MIMIC_CXR.dataset import CXRDataset
from MIMIC_CXR.utils import Configs as CXRConfigs

# Configs

In [6]:
@dataclass
class Configs(SharedConfigs):
    CXR_DATA_DIR = os.path.join("data", "MIMIC-CXR-JPG")
    CXR_VALID_LABELS_FILENAME = "valid.csv"
    CXR_FILENAMES = CXRConfigs.CXR_FILENAMES
    CHEXPERT_DATA_DIR = os.path.join("data", "CheXpert", "CheXpert-v1.0-small")
    CHEXPERT_DISEASE_TRAINED_MODELS_DIR = os.path.join("CheXpert", "disease_prediction", "trained_models")
    CHEXPERT_RACE_TRAINED_MODELS_DIR = os.path.join("CheXpert", "race_prediction", "trained_models")
    CHEXPERT_VALID_LABELS_FILENAME = "valid.csv"
    CHEXPERT_DEMO_FILENAME = "CHEXPERT DEMO.csv"
    DISEASE_ANNOTATIONS_COLUMNS = DiseaseConfigs.ANNOTATIONS_COLUMNS
    RACE_ANNOTATIONS_COLUMNS = RaceConfigs.ANNOTATIONS_COLUMNS
    NUM_DISEASE_CLASSES = DiseaseConfigs.NUM_CLASSES
    NUM_RACE_CLASSES = RaceConfigs.NUM_CLASSES
    CHEXPERT_RACE_DICT = RaceConfigs.RACE_DICT
    BATCH_SIZE = 4
    VALID_SIZE_DEBUG = 10**5

In [7]:
shared_utils.set_seed(Configs.SEED)

In [8]:
if torch.cuda.is_available():
    vprint(f"Memory info: {torch.cuda.mem_get_info()[0]/10e8:.1f} GB free GPU.", Configs)
else: 
    vprint(f"No GPU Memory.", Configs)

2022-07-23 21:16: Memory info: 8.5 GB free GPU.


In [9]:
valid_transform = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Disease Prediction 

## Validaiton Dataloaders

In [10]:
cxp_disease_valid_dataset = CheXpertDiseaseDataset(data_dir=Configs.CHEXPERT_DATA_DIR, 
                                                   labels_filename=Configs.CHEXPERT_VALID_LABELS_FILENAME,
                                                   transform=valid_transform)
cxp_disease_valid_dataset.df_labels = cxp_disease_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG]
cxp_disease_valid_dataloader = DataLoader(cxp_disease_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(cxp_disease_valid_dataset)

234

In [11]:
# cxr_disease_valid_dataset = CXRDataset.download_dataset(100, cxr_mode.Disease, Configs.CXR_DATA_DIR,
#                                                         Configs.CXR_VALID_LABELS_FILENAME, **Configs.CXR_FILENAMES,
#                                                         transform=valid_transform, target_transform=None)
cxr_disease_valid_dataset = CXRDataset(Mode.Disease, Configs.CXR_DATA_DIR, Configs.CXR_VALID_LABELS_FILENAME,
                                       transform=valid_transform)
cxr_disease_valid_dataset.df_labels = cxr_disease_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG]
cxr_disease_valid_dataloader = DataLoader(cxr_disease_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(cxr_disease_valid_dataset)

3000

## Pretrained Models 

In [12]:
_, _, files = next(os.walk(Configs.CHEXPERT_DISEASE_TRAINED_MODELS_DIR))
disease_trained_models = [os.path.join(Configs.CHEXPERT_DISEASE_TRAINED_MODELS_DIR, file) for file in files]
len(disease_trained_models)

1

In [13]:
disease_model = densenet121()
num_features = disease_model.classifier.in_features
disease_model.classifier = nn.Sequential(
    nn.Linear(num_features, num_features, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(in_features=num_features, out_features=Configs.NUM_DISEASE_CLASSES, bias=True)
)
disease_model.eval()
not disease_model.training

True

In [14]:
disease_model, results, _, _ = shared_utils.load_statedict(disease_model, disease_trained_models[0], Configs)
disease_model = to_gpu(disease_model)

2022-07-23 21:16: Loading model - CheXpert/disease_prediction/trained_models/2022_07_12-18_47__densenet121_aug__epoch-5__iter-12659__batch_size-16__trainLastLoss-0.3754__validAUC-0.8899.dict


## Predictions

In [15]:
df_res_disease = pd.DataFrame(columns=Configs.DISEASE_ANNOTATIONS_COLUMNS + ['Mean'])
df_res_disease

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean


In [16]:
cxp_disease_labels, cxp_disease_outputs = shared_utils.get_metric_tensors(disease_model, cxp_disease_valid_dataloader, Configs,
                                                  apply_on_outputs=lambda x: torch.sigmoid(x),
                                                  by_study=False, challenge_ann_only=None)
df_res_disease.loc['CXP'] = add_mean_to_list(shared_utils.auc_score(cxp_disease_labels, cxp_disease_outputs, per_class=True))

In [17]:
cxr_disease_labels, cxr_disease_outputs = shared_utils.get_metric_tensors(disease_model, cxr_disease_valid_dataloader, Configs,
                                                  apply_on_outputs=lambda x: torch.sigmoid(x),
                                                  by_study=False, challenge_ann_only=None)
df_res_disease.loc['CXR'] = add_mean_to_list(shared_utils.auc_score(cxr_disease_labels, cxr_disease_outputs, per_class=True))

In [18]:
df_res_disease.sort_values(by="Mean", ascending=False, inplace=True)
df_res_disease = df_res_disease.round(2)

In [19]:
df_res_disease

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
CXP,0.84,0.83,0.93,0.94,0.94,0.90
CXR,0.80,0.80,0.82,0.88,0.90,0.84


## Performance Per Protected Groups 

In [20]:
# race, age group, and gender to ChexPertDiseaseDatatset
cxp_demo_df = CheXpertRaceDataset.generate_race_dummies(pd.read_csv(os.path.join(Configs.CHEXPERT_DATA_DIR,
                                                                                 Configs.CHEXPERT_DEMO_FILENAME)),
                                                       'PRIMARY_RACE', Configs.CHEXPERT_RACE_DICT)
cxp_race_df = cxp_demo_df[['PATIENT', 'race'] + Configs.RACE_ANNOTATIONS_COLUMNS].drop_duplicates()
cxp_disease_df_labels = cxp_disease_valid_dataset.df_labels.merge(cxp_race_df, how='left',
                                                                                left_on='patient_id', right_on='PATIENT')
cxp_disease_df_labels.race.fillna('Other', inplace=True)
cxp_disease_df_labels['age'] = cxp_disease_df_labels.Age.apply(shared_utils.age_to_age_group)
cxp_disease_df_labels['gender'] = cxp_disease_df_labels.Sex
cxp_disease_df_labels.head(2)

,original_path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,img_path,patient_id,study,view,PATIENT,race,Asian,Black,Hispanic,White,age,gender
0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/home/student/MLH/debiasing-racial-effect-in-m...,patient64541,study1,view1_frontal.jpg,patient64541,White,0.0,0.0,0.0,1.0,70-90,Male
1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,/home/student/MLH/debiasing-racial-effect-in-m...,patient64542,study1,view1_frontal.jpg,patient64542,White,0.0,0.0,0.0,1.0,70-90,Male


In [21]:
cxr_disease_df_labels = cxr_disease_valid_dataset.df_labels.copy()
cxr_disease_df_labels.gender.replace({"M": "Male", "F": "Female"}, inplace=True)
cxr_disease_df_labels.head(2)

,subject_id,study_id,split,dicom_id,ethnicity,race,age,gender,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,folder_number,img_path
0,19859532,50453930,train,a5369d17-994f6cf1-dee57320-252febcc-6906f561,ASIAN,Asian,20-40,Female,0.0,0.0,0.0,0.0,0.0,19,data/MIMIC-CXR-JPG/physionet.org/files/mimic-c...
1,17719678,54809507,train,5d2b5624-28073582-14636856-8a3597b6-4cbc4bf3,ASIAN,Asian,20-40,Female,0.0,0.0,0.0,0.0,1.0,17,data/MIMIC-CXR-JPG/physionet.org/files/mimic-c...


In [22]:
shared_utils.auc_per_protected_group(cxp_disease_df_labels, Mode.Disease, Configs, cxp_disease_labels,
                                     cxp_disease_outputs, protected_groups=['race'])

Only one class present in y_true. ROC AUC score is not defined in that case.


,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
race,,,,,,
Asian,0.80,0.85,0.89,1.00,0.92,0.8920
Black,1.00,1.00,1.00,0.90,1.00,0.9800
Hispanic,1.00,0.75,1.00,NaN,1.00,0.9375
Other,0.78,0.84,0.93,0.91,0.98,0.8880
White,0.85,0.82,0.94,0.94,0.92,0.8940


In [23]:
shared_utils.auc_per_protected_group(cxp_disease_df_labels, Mode.Disease, Configs, cxp_disease_labels,
                                     cxp_disease_outputs, protected_groups=['gender'])

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
gender,,,,,,
Female,0.81,0.81,0.90,0.93,0.96,0.882
Male,0.85,0.85,0.96,0.94,0.92,0.904


In [24]:
shared_utils.auc_per_protected_group(cxp_disease_df_labels, Mode.Disease, Configs, cxp_disease_labels,
                                     cxp_disease_outputs, protected_groups=['age'])

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
age,,,,,,
20-40,0.92,0.89,0.94,1.00,0.90,0.930
40-70,0.84,0.82,0.94,0.92,0.94,0.892
70-90,0.77,0.83,0.93,0.94,0.92,0.878


In [25]:
shared_utils.auc_per_protected_group(cxr_disease_df_labels, Mode.Disease, Configs, cxr_disease_labels,
                                     cxr_disease_outputs, protected_groups=['race'])

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
race,,,,,,
Asian,0.81,0.81,0.83,0.88,0.91,0.848
Black,0.83,0.79,0.83,0.87,0.92,0.848
Hispanic,0.81,0.81,0.85,0.89,0.90,0.852
Other,0.77,0.79,0.78,0.87,0.84,0.810
White,0.78,0.81,0.80,0.88,0.92,0.838


In [26]:
shared_utils.auc_per_protected_group(cxr_disease_df_labels, Mode.Disease, Configs, cxr_disease_labels,
                                     cxr_disease_outputs, protected_groups=['gender'])

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
gender,,,,,,
Female,0.81,0.8,0.82,0.90,0.9,0.846
Male,0.79,0.8,0.81,0.85,0.9,0.830


In [27]:
shared_utils.auc_per_protected_group(cxr_disease_df_labels, Mode.Disease, Configs, cxr_disease_labels,
                                     cxr_disease_outputs, protected_groups=['age'])

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
age,,,,,,
20-40,0.88,0.83,0.89,0.91,0.94,0.890
40-70,0.79,0.79,0.85,0.88,0.87,0.836
70-90,0.71,0.73,0.71,0.82,0.87,0.768


In [28]:
shared_utils.auc_per_protected_group(cxr_disease_df_labels, Mode.Disease, Configs, cxr_disease_labels,
                                     cxr_disease_outputs, protected_groups=['gender', 'age','race'])

Only one class present in y_true. ROC AUC score is not defined in that case.


Atelectasis  Cardiomegaly  Consolidation  Edema  Pleural Effusion    Mean
gender age   race                                                                               
Female 20-40 Asian            0.93          0.83           0.95   0.93              0.94  0.9160
             Black            0.95          0.89           1.00   0.99              0.96  0.9580
             Hispanic         0.90          0.76           0.95   0.99              0.98  0.9160
             Other            0.87          0.91           0.87   0.93              0.81  0.8780
             White            0.86          0.91           0.92   0.91              0.98  0.9160
       40-70 Asian            0.83          0.84           0.76   0.86              0.93  0.8440
             Black            0.77          0.70           0.33   0.89              0.91  0.7200
             Hispanic         0.84          0.87            NaN   0.86              0.94  0.8775
             Other            0.73          0.74           0.92   0.88              0.72  0.7980
             White            0.72          0.72           0.83   0.91              0.86  0.8080
       70-90 Asian            0.69          0.75           0.72   0.89              0.85  0.7800
             Black            0.78          0.75           0.78   0.88              0.90  0.8180
             Hispanic         0.73          0.75           0.68   0.80              0.88  0.7680
             Other            0.72          0.75           0.62   0.84              0.83  0.7520
             White            0.68          0.69           0.92   0.86              0.91  0.8120
Male   20-40 Asian            0.91          0.87           0.91   0.87              0.97  0.9060
             Black            0.85          0.86           0.91   0.87              0.90  0.8780
             Hispanic         0.81          0.81           0.86   0.86              0.96  0.8600
             Other            0.82          0.73           0.80   0.83              0.90  0.8160
             White            0.91          0.79           0.84   0.91              0.95  0.8800
       40-70 Asian            0.78          0.82           0.90   0.90              0.94  0.8680
             Black            0.85          0.80           0.85   0.83              0.90  0.8460
             Hispanic         0.81          0.80           0.92   0.89              0.79  0.8420
             Other            0.73          0.73           0.81   0.90              0.81  0.7960
             White            0.80          0.83           0.75   0.84              0.92  0.8280
       70-90 Asian            0.66          0.72           0.72   0.74              0.82  0.7320
             Black            0.74          0.72           0.68   0.73              0.90  0.7540
             Hispanic         0.72          0.78           0.99   0.87              0.86  0.8440
             Other            0.69          0.74           0.67   0.78              0.85  0.7460
             White            0.72          0.68           0.69   0.80              0.88  0.7540

# Race Prediction

## Validation Dataloaders

In [29]:
cxp_race_valid_dataset = CheXpertRaceDataset(data_dir=Configs.CHEXPERT_DATA_DIR, demo_filename=Configs.CHEXPERT_DEMO_FILENAME, 
                                             labels_filename=Configs.CHEXPERT_VALID_LABELS_FILENAME, transform=valid_transform)
cxp_race_valid_dataset.df_labels = cxp_race_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG]
cxp_race_valid_dataloader = DataLoader(cxp_race_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(cxp_race_valid_dataset)

172

In [30]:
# cxr_disease_valid_dataset = CXRDataset.download_dataset(10, cxr_mode.Race, Configs.CXR_DATA_DIR,
#                                                         Configs.CXR_VALID_LABELS_FILENAME, **Configs.CXR_FILENAMES,
#                                                         transform=valid_transform, target_transform=None)
cxr_race_valid_dataset = CXRDataset(Mode.Race, Configs.CXR_DATA_DIR, Configs.CXR_VALID_LABELS_FILENAME,
                                    transform=valid_transform)
cxr_race_valid_dataset.df_labels = cxr_race_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG]
cxr_race_valid_dataloader = DataLoader(cxr_race_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(cxr_race_valid_dataset)

2400

## Pretrained Models 

In [31]:
_, _, files = next(os.walk(Configs.CHEXPERT_RACE_TRAINED_MODELS_DIR))
race_trained_models = [os.path.join(Configs.CHEXPERT_RACE_TRAINED_MODELS_DIR, file) for file in files]
race_model_versions = [p.split('__')[1] for p in race_trained_models]
len(race_trained_models)

8

In [32]:
models_dict = {}
for model_version, model_path in zip(race_model_versions, race_trained_models):
    model = densenet121()
    if "shallow" in model_version:
        shallow_denseblock = int(model_version.split('_')[2][10:])
        layer_offset = 3 + 2 * shallow_denseblock
        num_features = model.features[layer_offset].norm.num_features
        model = model.features[:layer_offset]
        classifier_module = nn.Sequential(
            nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),   
            nn.Flatten(start_dim=1),
            nn.Linear(in_features=num_features, out_features=num_features, bias=True),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=num_features, out_features=Configs.NUM_RACE_CLASSES, bias=True))
        model.add_module('classifier', classifier_module)
    else:
        num_features = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Linear(num_features, num_features, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=num_features, out_features=Configs.NUM_RACE_CLASSES, bias=True))
    model.eval()
    model, results, _, _ = shared_utils.load_statedict(model, model_path, Configs, device='cpu')
    models_dict[model_version] = model

2022-07-23 21:20: Loading model - CheXpert/race_prediction/trained_models/2022_07_19-15_29__densenet121_race_denseblock1_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.2805__validAUC-0.9178.dict
2022-07-23 21:20: Loading model - CheXpert/race_prediction/trained_models/2022_07_19-02_27__densenet121_race_denseblock2_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.2811__validAUC-0.9094.dict
2022-07-23 21:20: Loading model - CheXpert/race_prediction/trained_models/2022_07_18-10_11__densenet121_race_classifier_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.732__validAUC-0.6978.dict
2022-07-23 21:20: Loading model - CheXpert/race_prediction/trained_models/2022_07_21-13_21__densenet121_race_denseblock2_shallow__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.3791__validAUC-0.9075.dict
2022-07-23 21:20: Loading model - CheXpert/race_prediction/trained_models/2022_07_18-21_46__densenet121_race_denseblock3_freezed__epoch-9__iter-10266__batch_size-16__

## Predictions 

In [33]:
df_res_race = pd.DataFrame(columns=Configs.RACE_ANNOTATIONS_COLUMNS + ['Mean'])
df_res_race

,Asian,Black,Hispanic,White,Mean


In [ ]:
for model_version, model in tqdm(models_dict.items()):
    model = to_gpu(model)
    cxp_race_labels, cxp_race_outputs = shared_utils.get_metric_tensors(model, cxp_race_valid_dataloader, Configs,
                                                                        apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                                        by_study=False, challenge_ann_only=None)
    cxr_race_labels, cxr_race_outputs = shared_utils.get_metric_tensors(model, cxr_race_valid_dataloader, Configs,
                                                                        apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                                        by_study=False, challenge_ann_only=None)
    df_res_race.loc[f"CXP_{model_version}"] = add_mean_to_list(shared_utils.auc_score(cxp_race_labels, cxp_race_outputs, per_class=True))
    df_res_race.loc[f"CXR_{model_version}"] = add_mean_to_list(shared_utils.auc_score(cxr_race_labels, cxr_race_outputs, per_class=True))
    model.cpu()

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
df_res_race.sort_values(by="Mean", ascending=False, inplace=True)
df_res_race = df_res_race.round(2)

In [ ]:
df_res_race

## Performance Per Protected Groups

In [ ]:
# race, age group, and gender to ChexPertDiseaseDatatset
cxp_race_df_labels = cxp_race_valid_dataset.df_labels[['PATIENT', 'race', 'AGE_AT_CXR', 'GENDER'] + Configs.RACE_ANNOTATIONS_COLUMNS].drop_duplicates()
cxp_race_df_labels['age'] = cxp_race_df_labels.AGE_AT_CXR.apply(shared_utils.age_to_age_group)
cxp_race_df_labels['gender'] = cxp_race_df_labels.GENDER
cxp_race_df_labels.head(2)

In [ ]:
cxr_race_df_labels = cxr_race_valid_dataset.df_labels.copy()
cxr_race_df_labels.gender.replace({"M": "Male", "F": "Female"}, inplace=True)
cxr_race_df_labels.head(2)

In [ ]:
model = models_dict['densenet121_race']
model = to_gpu(model)
cxp_race_labels, cxp_race_outputs = shared_utils.get_metric_tensors(model, cxp_race_valid_dataloader, Configs,
                                                                    apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                                    by_study=False, challenge_ann_only=None)
cxr_race_labels, cxr_race_outputs = shared_utils.get_metric_tensors(model, cxr_race_valid_dataloader, Configs,
                                                                    apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                                    by_study=False, challenge_ann_only=None)

In [ ]:
shared_utils.auc_per_protected_group(cxp_race_df_labels, Mode.Race, Configs, cxp_race_labels,
                                     cxp_race_outputs, protected_groups=['gender'])

In [ ]:
shared_utils.auc_per_protected_group(cxp_race_df_labels, Mode.Race, Configs, cxp_race_labels,
                                     cxp_race_outputs, protected_groups=['age'])

In [ ]:
shared_utils.auc_per_protected_group(cxr_race_df_labels, Mode.Race, Configs, cxr_race_labels,
                                     cxr_race_outputs, protected_groups=['gender'])

In [ ]:
shared_utils.auc_per_protected_group(cxr_race_df_labels, Mode.Race, Configs, cxr_race_labels,
                                     cxr_race_outputs, protected_groups=['age'])

In [ ]:
shared_utils.auc_per_protected_group(cxr_race_df_labels, Mode.Race, Configs, cxr_race_labels,
                                     cxr_race_outputs, protected_groups=['gender', 'age'])